# 1. Import Packages

In [ ]:
%load_ext tensorboard
!pip install tensorflow==2.3


In [15]:
!pip install tensorflow_addons

In [1]:
!pip install tf-nightly

  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.1 requires flatbuffers<2,>=1.12, but you have flatbuffers 2.0 which is incompatible.
tensorflow-gpu 2.4.1 requires absl-py~=0.10, but you have absl-py 1.1.0 which is incompatible.
tensorflow-gpu 2.4.1 requires flatbuffers~=1.12.0, but you have flatbuffers 2.0 which is incompatible.
tensorflow-gpu 2.4.1 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow-gpu 2.4.1 requires tensorflow-estimator<2.5.0,>=2.4.0, but you have tensorflow-estimator 2.9.0 which is incompatible.


In [2]:
pip install --upgrade tensorflow

  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-nightly 2.10.0.dev20220701 requires flatbuffers>=2.0, but you have flatbuffers 1.12 which is incompatible.
tensorflow-gpu 2.4.1 requires absl-py~=0.10, but you have absl-py 1.1.0 which is incompatible.
tensorflow-gpu 2.4.1 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow-gpu 2.4.1 requires tensorflow-estimator<2.5.0,>=2.4.0, but you have tensorflow-estimator 2.9.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import tensorflow as tf
from PIL import Image
from matplotlib import pyplot as plt
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, ReLU, Dropout, MaxPooling2D
import tensorflow.keras.layers as tfl
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomContrast
from tensorflow.keras.utils import image_dataset_from_directory
import os
import numpy as np
import tensorflow_addons as tfa
from datetime import datetime
from tensorflow import keras
import sklearn
import tensorflow.keras.backend as kb
import PIL
from sklearn.metrics import multilabel_confusion_matrix
import seaborn as sns

/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


# 2. Preprocess

# Crop and Resize

In [ ]:
def crop_image(current_location, target_location, target_size=(224, 224)):
  image = Image.open(current_location)
  image_array = np.array(image)
  min_width = np.argwhere(image_array[712, :, :]> 10)[0,0]
  max_width = np.max(np.argwhere(image_array[712, :, :]> 10))
  cropped_image = image_array[:, min_width:max_width, :]
  new_image = Image.fromarray(cropped_image, 'RGB')
  new_image = new_image.resize(target_size)
  new_image.save(target_location)
  return new_image

## iterate over dataset (run only once)

In [ ]:
os.mkdir('/kaggle/working/Test')
os.mkdir('/kaggle/working/Eval')
os.mkdir('/kaggle/working/Train')

In [ ]:
for i in range(1, 641):
  current_location = f"../input/retinal-disease-classification/Test_Set/Test_Set/Test/{i}.png"
  target_location = f"/kaggle/working/Test/{i}.png"
  crop_image(current_location, target_location)
print('done!')
for i in range(1, 641):
  current_location = f"../input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/Validation/{i}.png"
  target_location = f"/kaggle/working/Eval/{i}.png"
  crop_image(current_location, target_location)
print('done!')
for i in range(1, 1921):
  current_location = f"../input/retinal-disease-classification/Training_Set/Training_Set/Training/{i}.png"
  target_location = f"/kaggle/working/Train/{i}.png"
  crop_image(current_location, target_location)
print('done!')

# 3. create labels

In [ ]:
def create_labels(csv_path : str, class_names):

  dataframe = pd.read_csv(csv_path)
  dataframe = dataframe[class_names]

  return dataframe.to_numpy()

In [ ]:
class_names = ['DR']
training_labels = create_labels('../input/retinal-disease-classification/Training_Set/Training_Set/RFMiD_Training_Labels.csv', class_names)
test_labels = create_labels('../input/retinal-disease-classification/Test_Set/Test_Set/RFMiD_Testing_Labels.csv', class_names)
validation_labels = create_labels('../input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv', class_names)

In [ ]:
plt.hist(training_labels.sum(axis=1), align='left', bins=[-1, 0, 1, 2, 3, 4, 5], rwidth=0.25)

In [ ]:
for y in [training_labels, validation_labels]:
  total = y.sum()
  total_labels = y.sum(axis=0)
  fractions = total_labels/total
  weights = 1/fractions

# 4. load data

In [ ]:
def parse_function(filename : str, label, channels=3):

    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_png(image_string, channels=channels)
    image_normalized = tf.cast(image_decoded, tf.float32) / 255.0

    return image_normalized, label

In [ ]:
batch_size = 8
sbf = 512
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
def create_dataset(filenames, labels, is_training=True):

    dir_list = os.listdir(filenames)
    new_dir_list = [ filenames + k for k in dir_list]
    dataset = tf.data.Dataset.from_tensor_slices((new_dir_list, labels))
    dataset = dataset.map(lambda x, y: parse_function(x, y), num_parallel_calls=AUTOTUNE)
        
    if is_training == True:
        dataset = dataset.cache()
        dataset = dataset.shuffle(buffer_size=sbf)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [ ]:
training_dataset = create_dataset('../input/retianaldiseasespreprocessed/Train/', training_labels)
validation_dataset = create_dataset('../input/retianaldiseasespreprocessed/Eval/', validation_labels)
test_dataset = create_dataset('../input/retianaldiseasespreprocessed/Test/', test_labels, is_training=False)

##run for new datset

In [63]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
directory = "../input/diabetic-retinopathy-preprocessed-dataset/Dataset/"
train_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='training',
                                             label_mode='categorical',
                                             seed=42)
validation_dataset = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='validation',
                                             label_mode='categorical',
                                             seed=42)

Found 13970 files belonging to 5 classes.
Using 11176 files for training.
Found 13970 files belonging to 5 classes.
Using 2794 files for validation.


In [64]:
for image, label in train_dataset.take(1):
    print(label)

tf.Tensor(
[[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]], shape=(32, 5), dtype=float32)


# 5. Augmentation

In [ ]:
def data_augmenter():

    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomRotation(0.4, fill_mode='constant')) 
    data_augmentation.add(RandomFlip())
    data_augmentation.add(RandomContrast(0.3))

    return data_augmentation

In [ ]:
"""data_augmentation = data_augmenter()
def prepare(ds, shuffle=False, augment=False):
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x), y), 
                num_parallel_calls=AUTOTUNE)

  return ds.prefetch(buffer_size=AUTOTUNE)"""

In [ ]:
data_augmentation = data_augmenter()
def prepare(ds, augment=False):
  if augment:
    #ds = ds.map(if_ds, num_parallel_calls=AUTOTUNE)
    labels = []
    images = []
    for x, y in ds:
      for k in range(batch_size):
        if y[k] == 1:
            labels.append(y[k])
            images.append(data_augmentation(tf.expand_dims(x[k], 0))[0])
    ds = tf.data.Dataset.from_tensor_slices((images, labels)) 
    ds = ds.batch(batch_size)
       
  return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
new_training_dataset = training_dataset
for i in range(3): 
  train_ds = prepare(training_dataset, augment=True)
  new_training_dataset = new_training_dataset.concatenate(train_ds)
new_training_dataset = new_training_dataset.shuffle(buffer_size=sbf)
new_training_dataset

In [ ]:
data_augmentation = data_augmenter()

for image, _ in training_dataset.take(1):
    plt.figure(figsize=(10, 10))
    
    first_image = image[0]
    #print(first_image.shape)
    plt.subplot(3, 3, 1)
    plt.imshow(first_image)
    for i in range(8):
        plt.subplot(3, 3, i + 2)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0])
        plt.axis('off')

In [ ]:
new_training_labels = []

for image, labels in new_training_dataset:
    for label in labels:
        new_training_labels.append(int(label))
plt.hist(new_training_labels, align='left', bins=[-1, 0, 1, 2, 3, 4, 5], rwidth=0.25)

# 6. Confusion Matrix

In [ ]:
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.
    
    Args:
       cm (array, shape = [n, n]): a confusion matrix of integer classes
       class_names (array, shape = [n]): String names of the integer classes
    """
    
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    
    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    
    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

In [ ]:
logdir = "/content/drive/MyDrive/Colab Notebooks/logs/image/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir = logdir, histogram_freq = 1)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def plot_to_image(figure):
    """
    Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call.
    """
    
    buf = io.BytesIO()
    
    # Use plt.savefig to save the plot to a PNG in memory.
    plt.savefig(buf, format='png')
    
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    
    # Use tf.image.decode_png to convert the PNG buffer
    # to a TF image. Make sure you use 4 channels.
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    
    # Use tf.expand_dims to add the batch dimension
    image = tf.expand_dims(image, 0)
    
    return image

In [ ]:
def log_confusion_matrix(epoch, logs):
    
    # Use the model to predict the values from the test_images.
    test_pred_raw = model.predict(test_dataset)
    
    test_pred = np.argmax(test_pred_raw, axis=1)
    
    # Calculate the confusion matrix using sklearn.metrics
    cm = sklearn.metrics.confusion_matrix(test_labels, test_pred)
    
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)
    
    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

# 7. Training

In [65]:
def block_inc(inputs):
  x1 = tfl.Conv2D(128, (1, 1), padding='same')(inputs)
  x1 = tfl.Conv2D(128, (3, 3), padding='same')(x1)
  x2 = tfl.Conv2D(64, (1, 1), padding='same')(inputs)
  x2 = tfl.Conv2D(64, (5, 5), padding='same')(x2)
  x3 = tfl.Conv2D(128, (1, 1), padding='same')(inputs)
  x3 = tfl.Conv2D(32, (7, 7), padding='same')(x3)
  x4 = tfl.MaxPool2D(strides=1, padding='same')(inputs)
  x = tfl.Concatenate()([x1, x2, x3, x4])
  x = tfl.MaxPool2D()(x)
  return x

In [66]:
def classification(pre_trained_model_name : str, input_shape: tuple =(224, 224, 3), scratch = True):
  
  if pre_trained_model_name == 'inception':
    pre_trained_model = InceptionV3(input_shape=input_shape, include_top=False, weights=None)
  
  elif pre_trained_model_name == 'resnet':
    pre_trained_model = ResNet50(input_shape=input_shape, include_top=False, weights=None)
  
  elif pre_trained_model_name == 'vgg':
    pre_trained_model = VGG19(input_shape=input_shape, include_top=False, weights=None)

  if scratch == False:
    for layer in pre_trained_model.layers:
      layer.trainable = False
  inputs = tf.keras.Input(shape=input_shape, name='Input')
  #x = block_inc(inputs)
  #x = block_inc(x)
  #x = block_inc(x)
  #x = block_inc(x)
  x = pre_trained_model(inputs)
  x = tfl.Flatten(name='Flatten')(x)
  x = tfl.Dense(64, activation='relu', name='1st_Fully_Connected')(x)
  x = tfl.Dense(32, activation='relu', name='2nd_Fully_Connected')(x)
  #x = tfl.Dropout(0.2)(x)
  x = tfl.Dense(5, activation='softmax', name='Output')(x)
  model = tf.keras.Model(inputs=inputs, outputs=x, name=f'Transfer_Learning_by_{pre_trained_model_name}')
  
  return model


In [67]:
model = classification('resnet')
model.summary()

Model: "Transfer_Learning_by_resnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 Flatten (Flatten)           (None, 100352)            0         
                                                                 
 1st_Fully_Connected (Dense)  (None, 64)               6422592   
                                                                 
 2nd_Fully_Connected (Dense)  (None, 32)               2080      
                                                                 
 Output (Dense)              (None, 5)                 165       
                                                                 
Total params: 30,012,549
Trainable para

In [68]:
def my_loss(y_true, y_pred):
  cost = tf.reduce_mean(kb.square(kb.dot(y_true, y_pred)))
  return cost

In [70]:
base_learning_rate = 0.01
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy', tfa.metrics.F1Score(5, average='macro', threshold=0.5, name='F1 Score')])

In [71]:
#callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2)

In [72]:
#cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
#%tensorboard --logdir "/content/drive/MyDrive/Colab Notebooks/logs/image"
history = model.fit(train_dataset, validation_data=validation_dataset, epochs=5)# callbacks=[tensorboard_callback, cm_callback])#, callbacks=[callback])

Epoch 1/5
 28/350 [=>............................] - ETA: 2:08:14 - loss: 54.7628 - accuracy: 0.3438 - F1 Score: 0.1369

In [ ]:
acc = [0.] + history.history['binary_accuracy']
val_acc = [0.] + history.history['val_binary_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

f1 = history.history['F1 Score']
f1_val = history.history['val_F1 Score']

plt.figure(figsize=(10, 10))
plt.subplot(3, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='upper right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(3, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy Loss')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')


plt.subplot(3, 1, 3)
plt.plot(f1, label='Training F1 Score')
plt.plot(f1_val, label='Validation F1 Score')
plt.legend(loc='upper right')
plt.ylabel('F1 Score')
#plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation F1 Score')
plt.xlabel('epoch')
plt.show()

In [ ]:
test_preds = model.predict(test_dataset, batch_size=128)

In [ ]:
def _get_streaming_metrics(prediction,label,num_classes):

    with tf.name_scope("test"):
        # the streaming accuracy (lookup and update tensors)
        accuracy,accuracy_update = tf.metrics.accuracy(label, prediction, 
                                               name='accuracy')
        # Compute a per-batch confusion
        batch_confusion = tf.confusion_matrix(label, prediction,
                                             num_classes=num_classes,
                                             name='batch_confusion')
        # Create an accumulator variable to hold the counts
        confusion = tf.Variable( tf.zeros([num_classes,num_classes], 
                                          dtype=tf.int32 ),
                                 name='confusion' )
        # Create the update op for doing a "+=" accumulation on the batch
        confusion_update = confusion.assign( confusion + batch_confusion )
        # Cast counts to float so tf.summary.image renormalizes to [0,255]
        confusion_image = tf.reshape( tf.cast( confusion, tf.float32),
                                  [1, num_classes, num_classes, 1])
        # Combine streaming accuracy and confusion matrix updates in one op
        test_op = tf.group(accuracy_update, confusion_update)

        tf.summary.image('confusion',confusion_image)
        tf.summary.scalar('accuracy',accuracy)

    return test_op,accuracy,confusion

In [ ]:
test_preds = test_preds > 0.5
cm = multilabel_confusion_matrix(test_labels, test_preds)

In [ ]:
for num in range(7):
  df_cm = pd.DataFrame(cm[num], index =['Positive', 'Negative'] ,
                    columns = ['Positive', 'Negative'])
  plt.figure(figsize = (40, 40))
  plt.subplot(4, 4, num + 1)
  sns.heatmap(df_cm, annot=True)

In [ ]:
y_preds